In [96]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import glob
import os
import librosa
from IPython.display import display

In [46]:
film_csv = "/Volumes/Seagate/natasha-diploma/content_data.csv"
trailers_csv = "/Volumes/Seagate/natasha-diploma/trailers.csv"

df1 = pd.read_csv(film_csv, index_col=None, header=0)
df2 = pd.read_csv(trailers_csv, index_col=None, header=0)
trailers_meta_info = pd.merge(df1, df2, left_on='id', right_on='content_id', how='left').drop('id', axis=1)

trailers_names = []
for i in range(0, len(trailers_meta_info)):
    url = trailers_meta_info.iloc[i]['url']
    trailers_names.append(url[url.rfind('/')+1:-4])
    
trailers_meta_info['trailers_name'] = trailers_names
trailers_meta_info.head()

,title_x,duration,release_date,restrict,orig_title,kinopoisk_id,imdb_rating,kinopoisk_rating,synopsis,categories,genres,properties,content_id,title_y,url,trailers_name
0,Философия Фила,5816.0,2019-04-05,18.0,Phil,960568,5.4,5.86,"Фил – стоматолог, страдающий от депрессии. Его...",{Фильмы},"{Драмы,Зарубежные,Комедии}","{ ""Место действия"" : [""США""], ""Качества"" : [""д...",60432,Трейлер (русский язык),http://file30.vcp.digitalaccess.ru/contents/a/...,403a42b604f504069c41d39c5a4fc2
1,Философия Фила,5816.0,2019-04-05,18.0,Phil,960568,5.4,5.86,"Фил – стоматолог, страдающий от депрессии. Его...",{Фильмы},"{Драмы,Зарубежные,Комедии}","{ ""Место действия"" : [""США""], ""Качества"" : [""д...",60432,Трейлер (английский язык),http://file51.vcp.digitalaccess.ru/contents/9/...,e45a4697e1e6be16f301f13197ffa4
2,Зубная фея,5123.0,2019-04-02,18.0,Tooth Fairy,1126022,2.2,0.00,Старая женщина начинает рассказывать легенду о...,{Фильмы},"{Зарубежные,Триллеры,Ужасы}","{ ""Место действия"" : [""не определено""], ""Качес...",64219,Трейлер (русский язык),http://file66.vcp.digitalaccess.ru/contents/1/...,11fe2dd4cccaf0567c878e19e1811a
3,Проснись,4667.0,2019-01-19,16.0,Wake Up,1069713,2.9,0.00,"Врач-психиатр исследует дневник девушки, котор...",{Фильмы},"{Боевики,Зарубежные,Криминал,Триллеры,Ужасы}","{ ""Аудитория"" : [""для взрослых"",""для подготовл...",65407,Трейлер (русский язык),http://file55.vcp.digitalaccess.ru/contents/b/...,a5b2dd3ed41aabb5733c7cfffff0bc
4,[4k] Адская кухня,6148.0,2019-08-09,16.0,The Kitchen,1117988,5.5,5.50,"Смотрите в 4К! Три жены ирландских гангстеров,...",{Фильмы},"{Боевики,Драмы,Зарубежные,Криминал,По комиксам}","{ ""Настроение"" : [""захватывающие"",""авантюрный""...",75149,Трейлер (дублированный),http://file11.vcp.digitalaccess.ru/contents/9/...,80870ec50515107ba7034ab758cebe


In [93]:
def feature_extraction(y, sr):
    hop_length = 256
    frame_length = 512
    
    energy = np.array([
        sum(abs(y[i:i+frame_length]**2))
        for i in range(0, len(y), hop_length)
    ])
    rmse = librosa.feature.rms(y, frame_length=frame_length, hop_length=hop_length, center=True)
    
    frames = range(len(energy))
    t = librosa.frames_to_time(frames, sr=sr, hop_length=hop_length)
    centro = librosa.feature.spectral_centroid(y, sr)[0]
    zero_crossings = librosa.zero_crossings(y, sr)
    zero_crossings_count = 0
    for i in range(len(zero_crossings)):
        if zero_crossings[i] == True:
            zero_crossings_count += 1
    tempo, beat_times = librosa.beat.beat_track(y, sr=sr, units='time')
    y_harmonic, y_percussive = librosa.effects.hpss(y, margin=(1.0, 5.0))
    Y = librosa.stft(y)
    Ydb = librosa.amplitude_to_db(abs(Y))
    spectral_rolloff = librosa.feature.spectral_rolloff(y, sr=sr)[0]
    mfcc = librosa.feature.mfcc(y, sr=sr)
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    
    return {
        'energy': np.mean(energy),
        'tempo': tempo,
        'centro': pd.Series(centro),
        'zero_crossings': pd.Series(zero_crossings),
        'zero_cross_count': zero_crossings_count,
        'beat_times': beat_times,
        'harmonic': np.mean(y_harmonic),
        'percussive': np.mean(y_percussive),
        'amplitude': np.mean(Ydb),
        'spectral_rolloff': np.mean(spectral_rolloff),
        'mfcc': np.mean(mfcc),
        'chroma_stft': np.mean(chroma_stft),
        'rmse': np.mean(rmse),
        'spec_bw': np.mean(spec_bw)
    }

In [98]:
img_info_folder = "/Volumes/Seagate/natasha-diploma/videoinfo"
# audio_info_folder = "/Volumes/Seagate/natasha-diploma/audioinfo"
audio_folder = "/Volumes/Seagate/natasha-diploma/audio_wav"
for i in range(0, len(trailers_meta_info)):
    name = trailers_meta_info.iloc[i]['trailers_name']
    img_info = img_info_folder + '/' + name + '.csv'
#     audio_info = audio_info_folder + '/' + name + '.csv'
    audio = audio_folder + '/' + name + '.wav'
    
    if os.path.exists(audio) and os.path.exists(img_info):
        x, sr = librosa.load(audio)
        audio_duration = librosa.get_duration(x) * 1000

        img_df = pd.read_csv(img_info, index_col=None, header=0)
        display(img_df.head())

        scenes = img_df['scene'].iloc[-1]
        dur_per_scene = audio_duration / (scenes + 1)
    
        if (os.path.exists(audio_info)):
            energy = []
            tempo = []
            centro = []
            zero_crossings = []
            zero_cross_count = []
            beat_times = []
            harmonic = []
            percussive = []
            amplitude = []
            spectral_rolloff = []
            mfcc = []
            chroma_stft = []
            rmse = []
            spec_bw = []
            newAudio = AudioSegment.from_wav(audio)
            
            for i in range(0, len(img_df)):   
                scene = int(img_df.iloc[i]['scene'])
                if (dur_per_scene * scene >= audio_duration):
                    break
                newAudio_temp = newAudio[dur_per_scene * scene:dur_per_scene * (scene + 1)]
                newAudio_temp.export('newSong.wav', format="wav")
                
                x_temp, sr_temp = librosa.load('newSong.wav')
                
                features = feature_extraction(x_temp, sr_temp)
                
                energy.append(features['energy'])
                tempo.append(features['tempo'])
                centro.append(features['centro'])
                zero_crossings.append(features['zero_crossings'])
                zero_cross_count.append(features['zero_cross_count'])
                beat_times.append(features['beat_times'])
                harmonic.append(features['harmonic'])
                percussive.append(features['percussive'])
                amplitude.append(features['amplitude'])
                spectral_rolloff.append(features['spectral_rolloff'])
                mfcc.append(features['mfcc'])
                chroma_stft.append(features['chroma_stft'])
                rmse.append(features['rmse'])
                spec_bw.append(features['spec_bw'])
            
            img_df['energy'] = energy
            img_df['tempo'] = tempo
            img_df['centro'] = centro
            img_df['zero_crossings'] = zero_crossings
            img_df['zero_cross_count'] = zero_cross_count
            img_df['beat_times'] = beat_times
            img_df['harmonic'] = harmonic
            img_df['percussive'] = percussive
            img_df['amplitude'] = amplitude
            img_df['spectral_rolloff'] = spectral_rolloff
            img_df['mfcc'] = mfcc
            img_df['chroma_stft'] = chroma_stft
            img_df['rmse'] = rmse
            img_df['spec_bw'] = spec_bw
            
            os.remove('newSong.wav')
            
            img_df.to_csv(img_info)
            display(img_df.head())
            

,Unnamed: 0,scene,img,colors,temperature,brightness,colorfulness,color_hist,energy,tempo,...,zero_cross_count,beat_times,harmonic,percussive,amplitude,spectral_rolloff,mfcc,chroma_stft,rmse,spec_bw
0,0,0,/Volumes/Seagate/natasha-diploma/trailers_mp4/...,[[ 0. 0. 0.]\n [ 0. 0. 0.]\n [ 0. 0. 0.]...,"[6500.7420431786531, 6500.7420431786531, 6500....","[0.0, 0.0, 0.0, 0.0, 0.0]","[0, 0, 0, 0, 0]",[ 1.],0.849772,50.666360,...,1,0 1.137778\ndtype: float64,-0.000011,-0.000015,-44.042313,477.080078,-13.422661,0.253869,0.031520,548.796989
1,1,0,/Volumes/Seagate/natasha-diploma/trailers_mp4/...,[[ 91.1734104 54.0867052 57.73410405]\n...,"[3098.7133096178086, 5790.4659133120122, 5184....","[35.754278590048742, 4.9233511586452758, 14.63...","[40.677106447727127, 19.534958564647194, 26.99...",[ 0.06843821 0.38377268 0.31305981 0.212528...,0.849772,50.666360,...,1,0 1.137778\ndtype: float64,-0.000011,-0.000015,-44.042313,477.080078,-13.422661,0.253869,0.031520,548.796989
2,2,0,/Volumes/Seagate/natasha-diploma/trailers_mp4/...,[[ 205.1150267 181.25507246 147.41800153]\n...,"[4497.2683226115814, 5768.5801305242931, 5941....","[80.437265371442209, 5.2838108010778724, 51.56...","[28.129106921457215, 16.118683901292563, 6.729...",[ 0.21356293 0.21087727 0.29708759 0.189200...,0.849772,50.666360,...,1,0 1.137778\ndtype: float64,-0.000011,-0.000015,-44.042313,477.080078,-13.422661,0.253869,0.031520,548.796989
3,3,1,/Volumes/Seagate/natasha-diploma/trailers_mp4/...,[[ 54.47333333 46.05333333 43.61333333]\n...,"[5060.6675595228471, 7295.3358095934273, 4153....","[21.362091503267973, 83.204208512673361, 56.64...","[19.93636029861706, 7.7790550638004348, 30.063...",[ 0.09986536 0.26995465 0.05598073 0.271726...,2.969211,184.570312,...,1,0 1.741497\n1 2.066576\n2 2.391655\nd...,-0.000022,-0.000269,-28.732180,1666.251221,-6.107763,0.270203,0.058254,1196.630635
4,4,1,/Volumes/Seagate/natasha-diploma/trailers_mp4/...,[[ 231.08355091 234.5926893 238.39425587]\n...,"[6796.4350358590154, 5648.0616239136516, 7345....","[93.487943480264164, 7.831675467681853, 83.956...","[3.0666447620612267, 16.454108300230008, 8.188...",[ 0.28555839 0.35228175 0.23404195 0.079350...,2.969211,184.570312,...,1,0 1.741497\n1 2.066576\n2 2.391655\nd...,-0.000022,-0.000269,-28.732180,1666.251221,-6.107763,0.270203,0.058254,1196.630635


,Unnamed: 0,scene,img,colors,temperature,brightness,colorfulness,color_hist,energy,tempo,...,zero_cross_count,beat_times,harmonic,percussive,amplitude,spectral_rolloff,mfcc,chroma_stft,rmse,spec_bw
0,0,0,/Volumes/Seagate/natasha-diploma/trailers_mp4/...,[[ 0. 0. 0.]\n [ 0. 0. 0.]\n [ 0. 0. 0.]...,"[6500.7420431786531, 6500.7420431786531, 6500....","[0.0, 0.0, 0.0, 0.0, 0.0]","[0, 0, 0, 0, 0]",[ 1.],0.849772,50.666360,...,1,[1.1377777777777778],-0.000011,-0.000015,-44.042313,477.080078,-13.422661,0.253869,0.031520,548.796989
1,1,0,/Volumes/Seagate/natasha-diploma/trailers_mp4/...,[[ 91.1734104 54.0867052 57.73410405]\n...,"[3098.7133096178086, 5790.4659133120122, 5184....","[35.754278590048742, 4.9233511586452758, 14.63...","[40.677106447727127, 19.534958564647194, 26.99...",[ 0.06843821 0.38377268 0.31305981 0.212528...,0.849772,50.666360,...,1,[1.1377777777777778],-0.000011,-0.000015,-44.042313,477.080078,-13.422661,0.253869,0.031520,548.796989
2,2,0,/Volumes/Seagate/natasha-diploma/trailers_mp4/...,[[ 205.1150267 181.25507246 147.41800153]\n...,"[4497.2683226115814, 5768.5801305242931, 5941....","[80.437265371442209, 5.2838108010778724, 51.56...","[28.129106921457215, 16.118683901292563, 6.729...",[ 0.21356293 0.21087727 0.29708759 0.189200...,0.849772,50.666360,...,1,[1.1377777777777778],-0.000011,-0.000015,-44.042313,477.080078,-13.422661,0.253869,0.031520,548.796989
3,3,1,/Volumes/Seagate/natasha-diploma/trailers_mp4/...,[[ 54.47333333 46.05333333 43.61333333]\n...,"[5060.6675595228471, 7295.3358095934273, 4153....","[21.362091503267973, 83.204208512673361, 56.64...","[19.93636029861706, 7.7790550638004348, 30.063...",[ 0.09986536 0.26995465 0.05598073 0.271726...,2.969211,184.570312,...,1,"[1.7414965986394557, 2.066575963718821, 2.3916...",-0.000022,-0.000269,-28.732180,1666.251221,-6.107763,0.270203,0.058254,1196.630635
4,4,1,/Volumes/Seagate/natasha-diploma/trailers_mp4/...,[[ 231.08355091 234.5926893 238.39425587]\n...,"[6796.4350358590154, 5648.0616239136516, 7345....","[93.487943480264164, 7.831675467681853, 83.956...","[3.0666447620612267, 16.454108300230008, 8.188...",[ 0.28555839 0.35228175 0.23404195 0.079350...,2.969211,184.570312,...,1,"[1.7414965986394557, 2.066575963718821, 2.3916...",-0.000022,-0.000269,-28.732180,1666.251221,-6.107763,0.270203,0.058254,1196.630635


,scene,img,colors,temperature,brightness,colorfulness,color_hist
0,0,/Volumes/HD710/natasha-diploma/trailers_mp4/5b...,[[ 0. 0. 0.]\n [ 0. 0. 0.]\n [ 0. 0. 0.]...,"[6500.7420431786531, 6500.7420431786531, 6500....","[0.0, 0.0, 0.0, 0.0, 0.0]","[0, 0, 0, 0, 0]",[ 1.]
1,0,/Volumes/HD710/natasha-diploma/trailers_mp4/5b...,[[ 0. 0. 0.]\n [ 0. 0. 0.]\n [ 0. 0. 0.]...,"[6500.7420431786531, 6500.7420431786531, 6500....","[0.0, 0.0, 0.0, 0.0, 0.0]","[0, 0, 0, 0, 0]",[ 1.]
2,0,/Volumes/HD710/natasha-diploma/trailers_mp4/5b...,[[ 0. 0. 0.]\n [ 0. 0. 0.]\n [ 0. 0. 0.]...,"[6500.7420431786531, 6500.7420431786531, 6500....","[0.0, 0.0, 0.0, 0.0, 0.0]","[0, 0, 0, 0, 0]",[ 1.]
3,1,/Volumes/HD710/natasha-diploma/trailers_mp4/5b...,[[ 248.75792507 248.77089337 248.76512968]\n...,"[6500.0907940320176, 6577.8383143754791, 6535....","[97.557213086963884, 16.045751633986928, 77.84...","[0.0052129489652324003, 2.8513238289205605, 1....",[ 0.50367477 0.00978733 0.12349537 0.353153...
4,1,/Volumes/HD710/natasha-diploma/trailers_mp4/5b...,[[ 0. 0. 0. ]\n [ 0.0...,"[6500.7420431786531, 6499.8782057809285, 6500....","[0.0, 0.0003270699439075046, 0.0, 0.0, 0.0]","[0, 0.0, 0, 0, 0]",[ 0.99836516 0.00163484]


KeyboardInterrupt: 